In [ ]:
import xarray as xr
import numpy as np

import cmocean
import matplotlib.pyplot as plt
import matplotlib.patches as mpl_patches
import matplotlib.lines as mpl_lines

import src_screening.model.accessor

In [ ]:
plt.style.use("paper")
plt.style.use("egu_journals")

In [ ]:
ds_hr = xr.open_dataset("../../data/raw/test/hr_nature_initial", engine="zarr").isel(time=24, ensemble=3)
ds_lr = xr.open_dataset("../../data/interim/template_lr.nc")

In [ ]:
def plt_bcs(ax, dataarray, cmap, coarse_val=1., vmin=0., vmax=1.):
    coarse_model = mpl_patches.Rectangle((-50000, -150000), 100000, 300000)
    coarse_model.set_facecolor(cmap(coarse_val))
    coarse_model.set_edgecolor("none")
    plt_coarse = ax.add_patch(coarse_model)
    plt_coarse.set_zorder(-10)

    im = ax.tripcolor(
        ds_hr.sinn.triangulation,
        dataarray,
        vmin=vmin,
        vmax=vmax,
        cmap=cmap,
        rasterized=True
    )

    border = mpl_patches.Rectangle((-20000, -100000), 40000, 200000)
    border.set_facecolor("none")
    border.set_edgecolor("black")
    plt_border = ax.add_patch(border)
    plt_border.set_zorder(1)

    ax.arrow(
        x=-25000, y=0, dx=10000, dy=0,
        length_includes_head=True, head_starts_at_zero=False,
        fc='k', ec='k', zorder=2,
        width=60, head_width=1500
    )
    ax.arrow(
        x=-15000, y=0, dx=-10000, dy=0,
        length_includes_head=True, head_starts_at_zero=False,
        fc='k', ec='k', zorder=2,
        width=60, head_width=1500
    )

    ax.arrow(
        x=25000, y=0, dx=-10000, dy=0,
        length_includes_head=True, head_starts_at_zero=False,
        fc='k', ec='k', zorder=2,
        width=60, head_width=1500
    )
    ax.arrow(
        x=15000, y=0, dx=10000, dy=0,
        length_includes_head=True, head_starts_at_zero=False,
        fc='k', ec='k', zorder=2,
        width=60, head_width=1500
    )

    ax.arrow(
        x=0, y=-105000, dx=0, dy=10000,
        length_includes_head=True, head_starts_at_zero=False,
        fc='k', ec='k', zorder=2,
        width=60, head_width=1500
    )
    ax.arrow(
        x=0, y=-95000, dx=0, dy=-10000,
        length_includes_head=True, head_starts_at_zero=False,
        fc='k', ec='k', zorder=2,
        width=60, head_width=1500
    )

    ax.arrow(
        x=0, y=105000, dx=0, dy=-10000,
        length_includes_head=True, head_starts_at_zero=False,
        fc='k', ec='k', zorder=2,
        width=60, head_width=1500
    )
    ax.arrow(
        x=0, y=95000, dx=0, dy=10000,
        length_includes_head=True, head_starts_at_zero=False,
        fc='k', ec='k', zorder=2,
        width=60, head_width=1500
    )

    ax.set_xlim(-30000, 30000)
    ax.set_ylim(-110000, 110000)
    return im

In [ ]:
fig = plt.figure(figsize=(3, 4), dpi=300)
gs = fig.add_gridspec(ncols=3, wspace=0.13)

plt.axis('off')

###### GRID ########
ax_grid = fig.add_subplot(gs[:, 0])
ax_grid.set_axis_off()
ax_grid.plot(
    ds_hr["Mesh2_node_x"][ds_hr["Mesh2_face_nodes"]].T, ds_hr["Mesh2_node_y"][ds_hr["Mesh2_face_nodes"]].T,
    c="firebrick", label="High-res", ls="-", lw=0.4, rasterized=True
)
ax_grid.plot(
    ds_lr["Mesh2_node_x"][ds_lr["Mesh2_face_nodes"]].T, ds_lr["Mesh2_node_y"][ds_lr["Mesh2_face_nodes"]].T,
    c="black", label="Low-res", ls="-", lw=0.6, rasterized=True
)

ax_grid.arrow(
    x=-20000, y=-102500, dx=40000, dy=0,
    length_includes_head=True, head_starts_at_zero=False,
    fc='k', ec='k',
    width=60, head_width=1500
)
ax_grid.annotate('40 km', xy = (0, -104500), va="top", ha="center")

ax_grid.arrow(
    x=-22500, y=-100000, dx=0, dy=200000,
    length_includes_head=True, head_starts_at_zero=False,
    fc='k', ec='k',
    width=60, head_width=1500
)
ax_grid.annotate('200 km', xy = (-23500, 0), va="center", ha="right", rotation=90)
    

border = mpl_patches.Rectangle((-20000, -100000), 40000, 200000)
border.set_facecolor("none")
border.set_edgecolor("black")
plt_border = ax_grid.add_patch(border)
plt_border.set_zorder(99)

ax_grid.set_xlim(-30000, 30000)  # 70000
ax_grid.set_ylim(-110000, 110000) # 280000

custom_lines = [
    mpl_lines.Line2D([0], [0], color="firebrick", lw=1, label="High-res"),
    mpl_lines.Line2D([0], [0], color="black", lw=1, label="Low-res")
]
legend = ax_grid.legend(handles=custom_lines, labels=["High-res", "Low-res"], loc='upper center', bbox_to_anchor=(0.5, 0.0))


####### Boundary conditions #########

# Stress
ax_stress = fig.add_subplot(gs[:, 1])
ax_stress.set_axis_off()
plt_stress = plt_bcs(ax_stress, ds_hr["stress_xy"], plt.get_cmap("cmo.balance"), coarse_val=0.5, vmin=-2500, vmax=2500)
ax_stress_bounds = ax_stress.get_position().bounds
cax = fig.add_axes([ax_stress_bounds[0]+0.05, 0.11, ax_stress_bounds[2]-0.1, 0.02])
cbar = fig.colorbar(plt_stress, orientation="horizontal", cax=cax)

# # Damage
ax_damage = fig.add_subplot(gs[:, 2])
ax_damage.set_axis_off()
plt_damage = plt_bcs(ax_damage, ds_hr["damage"], plt.get_cmap("cmo.ice"), coarse_val=1., vmin=0., vmax=1.)
ax_damage_bounds = ax_damage.get_position().bounds
cax = fig.add_axes([ax_damage_bounds[0]+0.05, 0.11, ax_damage_bounds[2]-0.1, 0.02])
cbar = fig.colorbar(plt_damage, orientation="horizontal", cax=cax)

# # Stress

ax_grid.text(0.5, 1.01, s="(a) grid", va="bottom", ha="center", transform=ax_grid.transAxes)
ax_stress.text(0.5, 1.01, s="(b) stress", va="bottom", ha="center", transform=ax_stress.transAxes)
ax_damage.text(0.5, 1.01, s="(c) damage", va="bottom", ha="center", transform=ax_damage.transAxes)

fig.savefig("figures/fig02_model_information.pdf", pad_inches=0., bbox_inches="tight")